# ⚡⚡ PyTorch Quickstart for the American Express - Default Prediction competition
This notebook shows how to define and train a Pytorch LSTM to leverages the time series structure of the data.

I expect Deep Learning models to dominate in this competition, so here's a simple LSTM architecture.

Parameters were not really tweaked so the baseline is improvable.

**Please consider upvoting if you find this work helpful. Don't fork without upvoting !**



In [7]:
!nvidia-smi

Sat Aug 20 05:07:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# 把Google Drive挂载到Colab里
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive/Colab/'):
        return '/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())
# %cd  .//drive/MyDrive/AMEX\ Project/notebooks

## Why PyTorch Lightning?

Lightning is simply organized PyTorch code. There's NO new framework to learn. For more details about Lightning visit the repo:

https://github.com/PyTorchLightning/pytorch-lightning

Run on CPU, GPU clusters or TPU, without any code changes



# Imports


In [10]:
# Build  enviroment 
%pip install pytorch-lightning
!pip install wandb
#login to tensorboard to save and 
%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
import pandas as pd
import gc
import numpy as np

# Torch and Sklearn
import pytorch_lightning as pl
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchmetrics import Metric
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
import torch.nn.functional as F
from torch import nn, Tensor
from torchmetrics.utilities import rank_zero_warn

# Typing 
from typing import Optional

In [12]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key='6cc6eb507a318cfdf65bc1fee3cd6852e6c8e44c')     #This is my personal user id.
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


To use your W&B account,
Go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. 
Get your W&B access token from here: https://wandb.ai/authorize


In [13]:
!ls
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(project="AMEX" ,log_model=True)

'1.1 EDA.ipynb'
'1.2.1 Fill missing values by RF (train set).ipynb'
'1.2.2 Fill missing values by RF (test set).ipynb'
'1.3 Data Processing Demo.ipynb'
'2.3.1 Feature Engineering-1.ipynb'
'2.3.2 Feature Engineering-2.ipynb'
'2.3.3 Feature Engineering-3.ipynb'
'2.3.4 Feature Engineering-4.ipynb'
'2.3.5 Feature Engineering-5.ipynb'
'2.3.6 Feature Engineering-6.ipynb'
'2.3.7 Feature Engineering-7.ipynb'
'2.3.8 Feature Engineering-8.ipynb'
'2.4 Interaction Feature Selection - Filter Method.ipynb'
'2.5 Wrapper Method.ipynb'
'3.2 LightGBM with Bayesian Optimisation.ipynb'
'3.6.1 MLP Bagging Training.ipynb'
'3.6.2 MLP Bagging Inference.ipynb'
'4.1 LSTM FE New.ipynb'
'4.2 amex-lstm-pytorch-lightning-training-cleaning-features.ipynb'
'4.3 amex-lstm-pytorch-lightning-inf-features.ipynb'
'4.4 amex-lstm-pytorch-lightning-inf-predict.ipynb'
 best_parameters.pkl
 build
 mycheckpoints
 submission_lightgbm.csv
 test_lightgbm.csv
 train_lightgbm.csv
 wandb


wandb: Currently logged in as: anony-moose-380572. Use `wandb login --relogin` to force relogin


In [14]:
# File system
train_file   = "../data/4-PreCompressed/FilledWithRandomForest/train_RandomForest(PreCompressed).parquet"
train_labels = "../data/1-original-data/train_labels.csv"
model_output_folder = '../models/experiment'

# Data
batch_size   = 256  #1028 to 512
# Number of GPU used - we only have access to one gpu , feels like num_workers = 1 will be the same
num_workers  = 4
epochs = 10

# Model 
# 輸入 x 的特徵數量
in_features=171
# 隱藏層 h 的特徵數量
hidden_dim=512      # 128 changed to 512
# LSTM 的循環層數 (Default=1)
num_layers=1  # 2 changed to 1
learning_rate=8e-3  # 1e-3 to 8e-3 or 1e-2
# bias: 若為 False，則 bias 初始化為 0 (Default=True)
# dropout: 若不為 0，則在每一層 LSTM 後都會接著 dropout (Default=0)
# bidirectional: 若為 True，則為雙向 LSTM (Default=False)

# Data

Reading and preprocessing the data

We read the data from @raddar's [dataset](http://https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format) that i splitted into chunks [dataset](https://www.kaggle.com/datasets/what5up/amex-data-integer-dtypes-100k-cid-per-chunk). @raddar has denoised the data so that we can achieve better results with his dataset than with the original competition csv files.

We also convert the dataframe into a 3D-tensor dataset as highlighted by Chris Deotte [here](https://www.kaggle.com/competitions/amex-default-prediction/discussion/327828) 



In [15]:
# Function for de-noise
def de_noise(train_df):
  column_list = train_df.columns.tolist()
  column_list = column_list[2:]
  list_with_noise = []
  for x in range(0,188):
    if (train_df[column_list[x]].fillna(-9999) % 1  == 0).all() == False :
        list_with_noise.append(column_list[x])
  train_df[list_with_noise] -= 0.005
  train_df[list_with_noise] = train_df[list_with_noise].round(2)
  print(train_df)
  return train_df

In [16]:
!ls

'1.1 EDA.ipynb'
'1.2.1 Fill missing values by RF (train set).ipynb'
'1.2.2 Fill missing values by RF (test set).ipynb'
'1.3 Data Processing Demo.ipynb'
'2.3.1 Feature Engineering-1.ipynb'
'2.3.2 Feature Engineering-2.ipynb'
'2.3.3 Feature Engineering-3.ipynb'
'2.3.4 Feature Engineering-4.ipynb'
'2.3.5 Feature Engineering-5.ipynb'
'2.3.6 Feature Engineering-6.ipynb'
'2.3.7 Feature Engineering-7.ipynb'
'2.3.8 Feature Engineering-8.ipynb'
'2.4 Interaction Feature Selection - Filter Method.ipynb'
'2.5 Wrapper Method.ipynb'
'3.2 LightGBM with Bayesian Optimisation.ipynb'
'3.6.1 MLP Bagging Training.ipynb'
'3.6.2 MLP Bagging Inference.ipynb'
'4.1 LSTM FE New.ipynb'
'4.2 amex-lstm-pytorch-lightning-training-cleaning-features.ipynb'
'4.3 amex-lstm-pytorch-lightning-inf-features.ipynb'
'4.4 amex-lstm-pytorch-lightning-inf-predict.ipynb'
 best_parameters.pkl
 build
 mycheckpoints
 submission_lightgbm.csv
 test_lightgbm.csv
 train_lightgbm.csv
 wandb


In [17]:
def load_train_df(train_file, train_labels , labelled = True): # edited labelled 
  # The original input data have split to 5 with only 400mb each(mentioned it have been denoised) .Not sure how they do it 
  # Not enough Ram as the data feed in is too big
    train = pd.read_parquet(train_file)
    # Specify S_2 is the data for time
    train['S_2'] = pd.to_datetime(train['S_2'])
    tmp = train[['customer_ID','S_2']].groupby('customer_ID').count()

    missing_cids = []
    for nb_available_rows in range(1, 14):
        cids = tmp[tmp['S_2'] == nb_available_rows].index.values
        batch_missing_cids = [cid for cid in cids for _ in range(13 - nb_available_rows)]
        missing_cids.extend(batch_missing_cids)

    train_part2 = train.iloc[:len(missing_cids)].copy()
    train_part2.loc[:] = np.nan
    train_part2['customer_ID'] = missing_cids

    train = pd.concat([train_part2, train])
    
    train = train.sort_values('customer_ID')
    # if , else is added
    if labelled == True:
      train_labels = pd.read_csv(train_labels)
      print(train_labels)
    else:
      df_etrain_labels = pd.DataFrame({'target' : []})
    train = pd.merge(train, train_labels, how='inner', on='customer_ID')
    
    train = train.sort_values('customer_ID')
    return train

train_df = load_train_df(train_file, train_labels)
print(train_df)
#train_df = de_noise(train_df)

                                              customer_ID  target
0       0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...       0
1       00000fd6641609c6ece5454664794f0340ad84dddce9a2...       0
2       00001b22f846c82c51f6e3958ccd81970162bae8b007e8...       0
3       000041bdba6ecadd89a52d11886e8eaaec9325906c9723...       0
4       00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...       0
...                                                   ...     ...
458908  ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...       0
458909  ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...       0
458910  ffff9984b999fccb2b6127635ed0736dda94e544e67e02...       0
458911  ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...       1
458912  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...       0

[458913 rows x 2 columns]
                                               customer_ID        S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-11-20   
12       0000099d6bd597052cdcda90ff

In [18]:
#print(train_df.head())
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

Is the GPU available? True


In [19]:
class DataModule(pl.LightningDataModule):
    # 
    def __init__(self, all_data: pd.DataFrame, batch_size: int = batch_size, num_workers: int = num_workers):
        super().__init__()
        self.all_data = all_data
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.sc = StandardScaler()

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        # All data comumns except customer_ID, target, and S_2 are features
        features = self.all_data.columns[2:-1]
        self.all_data[features] = self.sc.fit_transform(self.all_data[features])
        self.all_data[features] = self.all_data[features].fillna(0)
        
        # https://www.kaggle.com/competitions/amex-default-prediction/discussion/327828 !! Many Thanks @Chris Deotte for your sharing
        all_tensor_x = torch.reshape(torch.tensor(self.all_data[features].to_numpy()), (-1, 13, 171)).float()
        all_tensor_y = torch.tensor(self.all_data.groupby('customer_ID').first()['target'].to_numpy()).float()
        #Split data Test and Train data
        X_trainval, X_test, y_trainval, y_test = train_test_split(all_tensor_x, all_tensor_y, test_size=0.1, random_state=1)
        # Data Train(X_trainval, y_trainval) ; Data Test( X_test, y_test)
        X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, random_state=1)

        # TRAIN
        self.train_tensor = TensorDataset(X_train, y_train)
        # VAL
        self.val_tensor = TensorDataset(X_val, y_val)
        # TEST
        self.test_tensor = TensorDataset(X_test, y_test)

    def train_dataloader(self):
        return DataLoader(self.train_tensor, batch_size=self.batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_tensor, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_tensor, batch_size=self.batch_size, num_workers=self.num_workers)
    # Added
    def predict_dataloader(self):
        return DataLoader(self.test_tensor, batch_size=self.batch_size, num_workers=self.num_workers)

# Metrics : [Implementation Source](https://www.kaggle.com/code/rohanrao/amex-competition-metric-implementations)

In [20]:
## https://www.kaggle.com/code/rohanrao/amex-competition-metric-implementations

class AmexMetric(Metric):
    is_differentiable: Optional[bool] = False

    # Set to True if the metric reaches it optimal value when the metric is maximized.
    # Set to False if it when the metric is minimized.
    higher_is_better: Optional[bool] = True

    # Set to True if the metric during 'update' requires access to the global metric
    # state for its calculations. If not, setting this to False indicates that all
    # batch states are independent and we will optimize the runtime of 'forward'
    full_state_update: bool = True

    def __init__(self):
        super().__init__()
        
        self.add_state("all_true", default=[], dist_reduce_fx="cat")
        self.add_state("all_pred", default=[], dist_reduce_fx="cat")

        rank_zero_warn(
            "Metric `Amex` will save all targets and predictions in buffer."
            " For large datasets this may lead to large memory footprint."
        )

    def update(self, y_pred: torch.Tensor, y_true: torch.Tensor):
        
        y_true = y_true.double()
        y_pred = y_pred.double()
        
        self.all_true.append(y_true)
        self.all_pred.append(y_pred)
        
    def compute(self):
        y_true = torch.cat(self.all_true)
        y_pred = torch.cat(self.all_pred)
        # count of positives and negatives
        n_pos = y_true.sum()
        n_neg = y_pred.shape[0] - n_pos

        # sorting by descring prediction values
        indices = torch.argsort(y_pred, dim=0, descending=True)
        preds, target = y_pred[indices], y_true[indices]

        # filter the top 4% by cumulative row weights
        weight = 20.0 - target * 19.0
        cum_norm_weight = (weight / weight.sum()).cumsum(dim=0)
        four_pct_filter = cum_norm_weight <= 0.04

        # default rate captured at 4%
        d = target[four_pct_filter].sum() / n_pos

        # weighted gini coefficient
        lorentz = (target / n_pos).cumsum(dim=0)
        gini = ((lorentz - cum_norm_weight) * weight).sum()

        # max weighted gini coefficient
        gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

        # normalized weighted gini coefficient
        g = gini / gini_max
        
        return 0.5 * (g + d)


def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x == 0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()

    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x == 0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

## Model

In [21]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""

    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, device):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        #全连接层的输入
        self.fc1 = nn.Linear(hidden_dim, 100)
        #全连接层的输出
        self.fc2 = nn.Linear(100, output_dim)
        print('test')
        ### Test - 全连接层的输入与输出
        #self.fc = nn.Linear(hidden_dim, output_dim)
        self.device = device

    def forward(self, x):
        h0, c0 = self.init_hidden(x)         #????
        out, (_, _) = self.rnn(x, (h0, c0))  #????
        out = F.relu(self.fc1(out[:, -1, :]))
        #Any real value is taken in where the value is reduced between 0 and 1
        out = torch.sigmoid(self.fc2(out))
        return out

    def init_hidden(self, x):
        batch_size = x.size(0)
        #返回一个形状为为size,类型为torch.dtype，里面的每一个值都是0的tensor
        # The input of this function is a 3x3 size number
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        if torch.cuda.is_available():
            h0, c0 = h0.cuda(), c0.cuda()
        return h0, c0


class TsLstmLightning(pl.LightningModule):
    def __init__(self, in_features, hidden_dim, num_layers, learning_rate):
        super(TsLstmLightning, self).__init__()

        self.learning_rate = learning_rate

        self.train_amex_metric = AmexMetric()
        self.val_amex_metric   = AmexMetric()
        self.test_amex_metric = AmexMetric()    #Added

        self.model = LSTMClassifier(in_features, hidden_dim, num_layers, 1, device = self.device)

        self.num_parameters = count_parameters(self.model)
        
        print(f"Trainable params: {self.num_parameters:,}")

        self.loss_fn = nn.BCELoss(reduction="mean")

    def forward(self, x):
        res = self.model(x)
        return res

    def training_step(self, batch, batch_idx):
        X, target = batch
        preds = self(X)  # (batch_size, 1)
        preds = preds.squeeze(1)

        loss = self.loss_fn(preds, target)
        
        self.train_amex_metric.update(preds, target) 

        self.log_dict({'train_loss': loss, 'train_amex_metric': self.train_amex_metric }, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            X, target = batch
            preds = self(X)  
            preds = preds.squeeze(1)

            loss = self.loss_fn(preds, target)
            
            self.val_amex_metric.update(preds, target),

            self.log_dict({'val_loss': loss, 'val_amex_metric': self.val_amex_metric}, on_step=True, on_epoch=True, prog_bar=True, logger=True)

            return {'loss': loss}
# Start added test_step - testing
    def test_step(self, batch, batch_idx):
        with torch.no_grad():
            X, target = batch
            preds = self(X)  
            preds = preds.squeeze(1)

            loss = self.loss_fn(preds, target)
            
            self.test_amex_metric.update(preds, target),

            self.log_dict({'test_loss': loss, 'test_amex_metric': self.test_amex_metric}, on_step=True, on_epoch=True, prog_bar=True, logger=True)

            return {'loss': loss}
#End
    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        with torch.no_grad():
            X = batch[0]
            preds = self(X)
            return preds.detach().cpu()

    def configure_optimizers(self):
      #Using adam optimiser
      #As Adma is so far the most optimiser
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Training Loop 

In [ ]:
print(f"Train Shape: {train_df.shape}")
dm = DataModule(train_df, batch_size=batch_size)

model = TsLstmLightning(in_features=in_features, hidden_dim=hidden_dim, num_layers=num_layers, learning_rate=learning_rate)

#logger = TensorBoardLogger(model_output_folder, name=f"logs", default_hp_metric=True)

#checkpoint_callback = ModelCheckpoint(dirpath= "mycheckpoints", save_top_k=1, save_weights_only=True, save_last=False, verbose=True,
#                                      monitor='val_loss_epoch', mode='min')

checkpoint_callback = ModelCheckpoint(dirpath= "mycheckpoints", save_weights_only=True, save_last=True, verbose=True,
                                      monitor='val_loss_epoch', mode='min')
callbacks = [checkpoint_callback]

trainer = Trainer(
    gpus=[0] if torch.cuda.is_available() else None,
    max_epochs=epochs,
    benchmark=False,
    deterministic=True,
    callbacks=callbacks,
    logger= wandb_logger)

trainer.fit(model, dm)
#trainer.summary()
# For enable a test loop, test_step will be needed in TsLstmLightning()

Train Shape: (5965869, 174)
test
Trainable params: 1,454,281


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `Amex` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_ch

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 11624: 'val_loss_epoch' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 13077: 'val_loss_epoch' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 14530: 'val_loss_epoch' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
wandb: WARNING Artifacts logged anonymously cannot be claimed and expire after 7 days.
wandb: WARNING Artifacts logged anonymously cannot be claimed and expire after 7 days.


In [ ]:
!ls

'1.1 EDA.ipynb'
'1.2.1 Fill missing values by RF (train set).ipynb'
'1.2.2 Fill missing values by RF (test set).ipynb'
'1.3 Data Processing Demo.ipynb'
'2.3.1 Feature Engineering-1.ipynb'
'2.3.2 Feature Engineering-2.ipynb'
'2.3.3 Feature Engineering-3.ipynb'
'2.3.4 Feature Engineering-4.ipynb'
'2.3.5 Feature Engineering-5.ipynb'
'2.3.6 Feature Engineering-6.ipynb'
'2.3.7 Feature Engineering-7.ipynb'
'2.3.8 Feature Engineering-8.ipynb'
'2.4 Interaction Feature Selection - Filter Method.ipynb'
'2.5 Wrapper Method.ipynb'
'3.2 LightGBM with Bayesian Optimisation.ipynb'
'3.6.1 MLP Bagging Training.ipynb'
'3.6.2 MLP Bagging Inference.ipynb'
'4.1 LSTM FE New.ipynb'
'4.2 amex-lstm-pytorch-lightning-training-cleaning-features.ipynb'
'4.3 amex-lstm-pytorch-lightning-inf-features.ipynb'
'4.4 amex-lstm-pytorch-lightning-inf-predict.ipynb'
 best_parameters.pkl
 build
 mycheckpoints
 submission_lightgbm.csv
 test_lightgbm.csv
 train_lightgbm.csv
 wandb


# Train Monitoring


In [ ]:
#Using wandb now

# Possible Nexts steps


1. Have a best handling of missiing values in the data ( Ex : Do not drop customers that don't have 13 records,  do not fill N/A with 0, Replace previous -1 values with N/A, ...)
2. Enhance the model (More LSTM / 1D CNN / Transformers / Param optimisation / ... ) 
3. Have a better understanding of the predictive features = Feature engineering ( feature selection or permutation feature importance, for instance.)
4. (1. + 2.--> Transformer Unsupervised training with Times Series) https://arxiv.org/abs/2010.02803
5. Model ensemble with other classification techniques
6. Enable Cross-validation (Stratified K-Fold, ...)


## TO DO

Getting confused with the format of dataset feed in the model

*   Getting confused with the format of dataset feed in the model (using MNIST()function and utils.data.DataLoader()???)
*   Need to add in predict_dataloader() in the TsLstmLightning() ?
*   Use tuner() for part of hyperparameter tuning 



In [ ]:
# default used by the Trainer
#trainer = Trainer(weights_save_path=os.getcwd())

# save to your custom path
#trainer = Trainer(weights_save_path="my/path")

# Using wandb to store weight
#def predict_data():
#  predict_df = 
#  features = self.all_data.columns[2:-1]
#  self.all_data[features] = self.sc.fit_transform(self.all_data[features])
#  self.all_data[features] = self.all_data[features].fillna(0)
#        
  # https://www.kaggle.com/competitions/amex-default-prediction/discussion/327828 !! Many Thanks @Chris Deotte for your sharing
#  all_tensor_x = torch.reshape(torch.tensor(self.all_data[features].to_numpy()), (-1, 13, 188)).float()
#  all_tensor_y = torch.tensor(self.all_data.groupby('customer_ID').first()['target'].to_numpy()).float()

#trainer.test(datamodule=dm)

In [ ]:
#test_file   = "./input/amex-data-integer-dtypes-100k-cid-per-chunk/train_chunk_0.parquet"
#test_df = pd.read_parquet(test_file)
#model.eval()
#dm_predict = DataModule(test_df, batch_size=batch_size)
#data_loader = DataLoader(test_df)
##predictions = trainer.predict(model, data_loader)
#trainer.test(datamodule=dm_predict)

In [ ]:
#test_file   = "./input/amex-data-integer-dtypes-100k-cid-per-chunk/test_chunk_0.parquet"
#test_df = pd.read_parquet(test_file)
#test_df['target']= np.zeros
#print(len(test_df))
#print(test_df.head())
#print(torch.from_numpy(test_df).type(torch.Tensor))
#trainer.predict(test_df)